# XAUUSD

In [240]:
import quandl
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm
import os

import warnings
import logging

In [132]:
xau_ratios = [
    "WGC/GOLD_DAILY_USD"
#     "WGC/GOLD_DAILY_EUR",
#     "WGC/GOLD_DAILY_TRY",
#     "WGC/GOLD_DAILY_JPY",
#     "WGC/GOLD_DAILY_GBP",
#     "WGC/GOLD_DAILY_CAD",
#     "WGC/GOLD_DAILY_CHF",
#     "WGC/GOLD_DAILY_VND",
#     "WGC/GOLD_DAILY_KRW",
#     "WGC/GOLD_DAILY_RUB",
#     "WGC/GOLD_DAILY_AUD",
]

economic_indc = []

### Quandl data terms

Anyone seeking to use this code must first apply for an account with [Quandl](https://www.quandl.com) in order to receive an valid authetitciation key.

In [39]:
DIR_NAME = os.path.abspath(os.path.join(os.getcwd(), '..'))
FILEPATH = os.path.join(DIR_NAME, "auth.txt")

with open(FILEPATH, "r") as f:
    authtoken = f.read();

# Retrieve Data

In [340]:
xau_df_dict = {}


for ratio in tqdm(xau_ratios):
    name = ratio.lower().replace("/", "_")

    # get the ratio dataframe
    df = quandl.get(ratio, authtoken=authtoken, start_date = "1979-01-01")
    df.columns = ["price"]
    
    # check for missing business days 
    freq = pd.infer_freq(df.index)
    
    if freq != "B":
        warnings.warn("Business day")
    
    xau_df_dict[name] = df


100%|██████████| 1/1 [00:03<00:00,  3.25s/it]

## Volatility

In [341]:
annualization_factor = 252.
window_size = [5, 20, 60, 120, 252]

for ratio, df in tqdm(xau_df_dict.items()):
    start_date, end_date = df.index[0], df.index[-1]
    full_range = pd.date_range(start_date, end_date, freq = "B")
    
    if not np.array_equal(df.index, pd.date_range(start_date, end_date, freq="B")):
        logging.warning("\n{} is missing business days".format(ratio))

    for window in window_size:
        df['{}d_market_vol'.format(window)] = np.sqrt(
            (annualization_factor/window) * df['price'].rolling(window).var(ddof=0))


100%|██████████| 1/1 [00:00<00:00,  2.62it/s]

In [342]:
features = [
    "FRED/T10Y2Y",
    "ML/AAAEY",
    "ML/CCCY",
    "RATEINF/INFLATION_USA",
]

In [343]:
for ratio, df in xau_df_dict.items():
    for feature in features:
        col_name = feature.lower().replace('/', '_')
        
        # get quandl features. `end_date` is set to df.index[-1] to match the price data
        data = quandl.get(feature, authtoken=authtoken, start_date = "1979-01-01", end_date = df.index[-1])
        start_date, end_date = data.index[0], data.index[-1]

        # Some features contain missing data. To best simulate how the data would be ingested
        # realtime, the current value is forward filled. This achieved by resampling.
        if not np.array_equal(data.index, pd.date_range(start_date, end_date, freq="B")):
            data = data.resample("B").ffill()
            logging.warning("\n\t{} is missing business days".format(feature))
                    
        df[col_name] = data

	FRED/T10Y2Y is missing business days
	ML/AAAEY is missing business days
	ML/CCCY is missing business days
	RATEINF/INFLATION_USA is missing business days


## Technical Indicators

In [344]:
import talib

technical_indicators= {
    "MACD" : ("macd", "macdsignal", "macdhist"),
    "STOCHRSI" : ("fastk", "fastd"),
    "MOM" : ("real")
}

for ratio, df in xau_df_dict.items():
    
    # talib requires market price data. starting price of $1 is taken
    # since absolute values are not important (preprocess scaling)
    price = df['price'].values

    for indicator, indicator_type in technical_indicators.items():
        # Return the result for each indicator
        result = getattr(talib, indicator)(price)

        if isinstance(result, np.ndarray):
            df[indicator.lower()] = result
        else:
            for f, r in zip(indicator_type, result):
                df["{}_{}".format(indicator.lower(), f)] = r

,price,5d_market_vol,20d_market_vol,60d_market_vol,120d_market_vol,252d_market_vol,fred_t10y2y,ml_aaaey,ml_cccy,rateinf_inflation_usa,macd_macd,macd_macdsignal,macd_macdhist,stochrsi_fastk,stochrsi_fastd,MOM
Date,,,,,,,,,,,,,,,,
1979-01-01,226.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-02,226.80,NaN,NaN,NaN,NaN,NaN,-0.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-03,218.60,NaN,NaN,NaN,NaN,NaN,-0.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-04,223.15,NaN,NaN,NaN,NaN,NaN,-0.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-05,225.50,21.053213,NaN,NaN,NaN,NaN,-0.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-08,223.10,19.867376,NaN,NaN,NaN,NaN,-0.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-09,224.00,16.361682,NaN,NaN,NaN,NaN,-0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-10,220.70,11.067618,NaN,NaN,NaN,NaN,-0.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-11,220.65,13.414476,NaN,NaN,NaN,NaN,-0.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
